# Q3. (30 points)
For both GCM and RMC, show empirically using the dataset I've shared that both models assume exchangeability of data,
viz. the order in which data enters the model does not affect the category labels of the model for any given subset of data. 

In [9]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [10]:
X_df = pd.read_csv("X.csv", names=["weight", "height", "label"])
y_df = pd.read_csv("y.csv", names=["weight", 'height'])

X_features = X_df.iloc[:, :2].values 
X_labels = X_df.iloc[:, 2].values
y_features = y_df.values


In [11]:
def calculate_similarity(test_point, train_features, train_labels, alpha_weight=2, alpha_height=1, beta=1):
    # Define attention weights for each training point
    alpha = np.array([alpha_weight, alpha_height])

    # Calculating similarity for each training point
    similarities = []
    for i, exemplar in enumerate(train_features):
        # Computing distance
        distance = np.sum(alpha * np.abs(exemplar - test_point))
        
        # Computing similarity
        similarity = np.exp(-beta * distance)
        
        # Appending similarity along with its label
        similarities.append((similarity, train_labels[i]))

    # Aggregating similarities by category
    small_sim = sum(similar for similar, label in similarities if label == 1)
    average_sim = sum(similar for similar, label in similarities if label == 2)
    large_sim = sum(similar for similar, label in similarities if label == 3)

    # Apply politeness bias: reduce the similarity weight for "large" and increase for "average"
    large_sim *= 0.8  
    average_sim *= 1.2  # Increase similarity weight for "average"

    # Choose category with highest similarity score
    similarities_dict = {1: small_sim, 2: average_sim, 3: large_sim}
    return max(similarities_dict, key=similarities_dict.get)

def likelihood(weight, height, category, params):
    # Calculating likelihood for weight and height
    weight_likelihood = norm.pdf(weight, params[category]['weight_mean'], params[category]['weight_std'])
    height_likelihood = norm.pdf(height, params[category]['height_mean'], params[category]['height_std'])
    
    # Adjusting likelihood based on weight's higher importance
    likelihood = (weight_likelihood ** 0.7) * (height_likelihood ** 0.3)
    return likelihood

def posterior_prob(weight, height, params):
    # Calculating posterior probability for each category
    posterior_probs = {}
    categories = ['small', 'average', 'large']
    cat_labels = {category: i + 1 for i, category in enumerate(categories)}
    
    for category in categories:
        cat_label = cat_labels[category]
        posterior_probs[cat_label] = likelihood(weight, height, category, params)
    
    # Normalizing the probabilities so that they sum to 1
    total_posterior = sum(posterior_probs.values())
    for cat_label in posterior_probs:
        posterior_probs[cat_label] /= total_posterior
    
    return posterior_probs

def predict_cat_label(posterior_probs):
    # Returning the category with the highest posterior probability
    return max(posterior_probs, key=posterior_probs.get)

def predict_test_instances(y_test, params):
    # Predicting labels for y.csv
    y_predictions = []
    
    for i, test_instance in enumerate(y_test):
        weight, height = test_instance
        posterior_probs = posterior_prob(weight, height, params)
        
        # Finding the category with the highest posterior probability
        predicted_label = predict_cat_label(posterior_probs)
        
        y_predictions.append(predicted_label)
        # print(f"Test instance {i + 1}\nWeight = {weight}, Height = {height}, Predicted label = {predicted_label}\n")
    
    return y_predictions


In [ ]:
def gcm_exchangeability_test(X_features, X_labels, y_features):
    # Original predictions
    original_predictions = [calculate_similarity(test_point, X_features, X_labels) for test_point in y_features]
    
    # Shuffle training data
    shuffled_X = np.concatenate((X_features, X_labels.reshape(-1, 1)), axis=1)
    np.random.shuffle(shuffled_X)
    shuffled_features = shuffled_X[:, :2]
    shuffled_labels = shuffled_X[:, 2]
    
    # Predictions after shuffling
    shuffled_predictions = [calculate_similarity(test_point, shuffled_features, shuffled_labels) for test_point in y_features]
    
    # Compare predictions
    return np.array_equal(original_predictions, shuffled_predictions)

def rmc_exchangeability_test(X, y_test, params):
    # Original predictions
    original_predictions = predict_test_instances(y_test, params)
    
    # Shuffle training data
    np.random.shuffle(X)
    
    # Recalculate parameters
    weights = X[:, 0]
    heights = X[:, 1]
    labels = X[:, 2]
    params = {
        'small': {
            'weight_mean': np.mean(X[labels == 1, 0]),
            'weight_std': np.std(X[labels == 1, 0]),
            'height_mean': np.mean(X[labels == 1, 1]),
            'height_std': np.std(X[labels == 1, 1])
        },
        'average': {
            'weight_mean': np.mean(X[labels == 2, 0]),
            'weight_std': np.std(X[labels == 2, 0]),
            'height_mean': np.mean(X[labels == 2, 1]),
            'height_std': np.std(X[labels == 2, 1])
        },
        'large': {
            'weight_mean': np.mean(X[labels == 3, 0]),
            'weight_std': np.std(X[labels == 3, 0]),
            'height_mean': np.mean(X[labels == 3, 1]),
            'height_std': np.std(X[labels == 3, 1])
        }
    }
    
    # Predictions after shuffling
    shuffled_predictions = predict_test_instances(y_test, params)
    
    # Compare predictions
    return np.array_equal(original_predictions, shuffled_predictions)


In [14]:
def main():
    X = pd.read_csv('X.csv', header=None).values
    y_test = pd.read_csv('y.csv', header=None).values
    
    # GCM exchangeability test
    gcm_result = gcm_exchangeability_test(X_features, X_labels, y_features)
    print(f"GCM exchangeability test: {gcm_result}")
    
    # RMC exchangeability test
    params = {
        'small': {
            'weight_mean': np.mean(X[X[:, 2] == 1, 0]),
            'weight_std': np.std(X[X[:, 2] == 1, 0]),
            'height_mean': np.mean(X[X[:, 2] == 1, 1]),
            'height_std': np.std(X[X[:, 2] == 1, 1])
        },
        'average': {
            'weight_mean': np.mean(X[X[:, 2] == 2, 0]),
            'weight_std': np.std(X[X[:, 2] == 2, 0]),
            'height_mean': np.mean(X[X[:, 2] == 2, 1]),
            'height_std': np.std(X[X[:, 2] == 2, 1])
        },
        'large': {
            'weight_mean': np.mean(X[X[:, 2] == 3, 0]),
            'weight_std': np.std(X[X[:, 2] == 3, 0]),
            'height_mean': np.mean(X[X[:, 2] == 3, 1]),
            'height_std': np.std(X[X[:, 2] == 3, 1])
        }
    }
    # RMC exchangeability test
    rmc_result = rmc_exchangeability_test(X, y_test, params)
    print(f"RMC exchangeability test: {rmc_result}")

if __name__ == "__main__":
    main()

GCM exchangeability test: True
RMC exchangeability test: True


### Results Explanation
##### GCM (Generalized Context Model) Exchangeability Test: True
This indicates that the GCM's categorization predictions remain unchanged when the training data is shuffled. This suggests:
1. The model's attention weights and similarity calculations are robust.
2. The model's performance is consistent, regardless of the training data order.

##### RMC (Rational Model of Categorization) Exchangeability Test: True
This result implies that the RMC's categorization predictions remain consistent when the training data is shuffled.
Model parameters (e.g., means and standard deviations) are recalculated.
This suggests:
1. The model's likelihood calculations and posterior probability estimates are stable.
2. The model's performance is reliable and insensitive to training data order.